In [56]:
#read needed params from cdf
def read_cdf(name, folder): 
    import cdflib
    import numpy as  np
    import os
    cdf_file = cdflib.CDF(f'{os.path.join("files", folder, name)}')
    cdf_file.cdf_info()
    freq = np.array(cdf_file.varget("freq_vlf"))

    ch1 = np.array(cdf_file.varget('ch1')) #ampl
    ch2 = np.array(cdf_file.varget('ch2')) #ampl

    epoch = np.array(cdf_file.varget('epoch_vlf')) #time
    time_bool =  epoch != epoch[0]
    epoch = epoch[time_bool]
    
    return ch1, freq, epoch, time_bool 

In [8]:
def from_epoch_to_time(epoch):
    import time
    import datetime
    import numpy as  np
    T = []
    T_sec = []
    for i in epoch:
        a = datetime.datetime.utcfromtimestamp(i//1e9)
        T.append(a.time().strftime('%H:%M'))
        T_sec.append(a.hour*3600 + a.minute*60 +a.second )
    try:
        T_sec_diff = (np.diff(T_sec))
        ind_midnight = int(np.where(T_sec_diff<0)[0])
        T_sec = T_sec[0:ind_midnight]
    except TypeError:
        ind_midnight = -99
    return T, T_sec, ind_midnight

In [10]:
def filtration(fr_max, ch1, freq, epoch, time_bool, ind_midnight):
    #remove freq more then fr_max and repeated values in time
    import numpy as  np


    freq_bool = freq<fr_max
    freq_filtr = freq[freq_bool]

    fr_bool =  np.array([freq_bool])
    tm_bool =  np.array([time_bool])

    filtr = tm_bool.transpose().dot(fr_bool)

    ch1_filtr = ch1[filtr].reshape((len(epoch)), len(freq_filtr))
    if ind_midnight!=-99:
        ch1_filtr = ch1_filtr[0:ind_midnight][:]
    return freq_filtr, ch1_filtr
    # ch1_filtr = ch1[time_bool][freq_bool].reshape((len(epoch)), len(freq_filtr)) 
    # print(ch1_filtr)

In [13]:
def plot_save_cdf(epoch, T, T_sec, freq_filtr, ch1_filtr, name, path_dir):
#plot contourf
    import numpy as  np
    import matplotlib.pyplot as plt
    import datetime
    import os
    plt.ioff()
    plt.figure(dpi = 100)
    plt.title(f'{datetime.datetime.utcfromtimestamp(epoch[0]//1e9).date()}')
    cs = plt.contourf(T_sec, freq_filtr, ch1_filtr.transpose())
    cbar = plt.colorbar(cs)
    plt.xticks(ticks=T_sec[0:(len(T_sec)-1):(len(T_sec)//7)], labels=T[0:(len(T_sec)-1):(len(T_sec)//7)])
    plt.xlabel('Time, UTC')
    plt.ylabel('Frequency, Hz')
    plt.grid(color = 'w')
    path = os.path.join(path_dir, f'{name}.jpg')
    plt.savefig(path)